In [1]:
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from collections import namedtuple
from typing import List, Callable
import optuna

In [2]:
class EstimatorOpt:
    def __init__(self, estimator, hyper_param_cv_callable):
        self.hyper_cv = hyper_param_cv_callable
        self.estimator = estimator
        
    def inner(inner_cv):
        return self.hyper_cv(estimator=self.estimator, cv=inner_cv)
    
    def nested(X, y, inner_cv, outer_cv):
        clf = self.hyper_cv(inner_cv)
        score = cross_val_score(clf, X=X, y=y, cv=outer_cv)
        return self.estimator, score

In [4]:
def k_fold_nested(estimator_opt: EstimatorOpt, split_strategy: Callable, X, y):
    inner_cv = split_strategy()
    outer_cv = split_strategy()
    estimator, score = estimator_opt.nested(X, y, inner_cv, outer_cv)
    return estimator, score

def k_fold_nested_model_comparison(estimator_opts: List[EstimatorOpt], split_strategy: Callable, X, y):
    inner_cv = split_strategy()
    outer_cv = split_strategy()
    results = [estimator_opt.nested(X, y, outer_cv, inner_cv) for estimator_opt in estimator_opts]
    return results

def grid_callable(param_grid):
    return lambda estimator, cv: GridSearchCV(estimator=estimator, param_grid=param_grid, cv=cv)

def random_search_callable(param_distr):
    return lambda estimator, cv: RandomizedSearchCV(estimator=estimator, param_distr=param_distr, cv=cv)

def optuna_search_callable(param_distr):
    return lambda estimator, cv: optuna.integration.OptunaSearchCV(estimator=estimator, param_distributions=param_distr, cv=cv)

def k_fold_strategy(n_splits, shuffle, random_state):
    return lambda: KFold(n_splits, shuffle, random_state)

def stratified_k_fold_strategy(n_splits, shuffle, random_state):
    return lambda: StratifiedKFold(n_splits, shuffle, random_state)